# LCEL接口
- 输入格式
- 输出格式
- 8种不同的接口方式

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-3.5-turbo",
)
prompt = ChatPromptTemplate.from_template("给我讲一个关于{topic}的笑话")
chain = prompt | model

## input schema

In [ ]:
chain.input_schema.schema()
# {'title': 'PromptInput',
#  'type': 'object',
#  'properties': {'topic': {'title': 'Topic', 'type': 'string'}}}

prompt.input_schema.schema()

model.input_schema.schema()

## Output Schema

In [ ]:
chain.output_schema.schema()

## 输出

In [ ]:
# Stream（流式）
for s in chain.stream({"topic": "熊"}):
    print(s.content, end="", flush=True)

# Invoke
chain.invoke({"topic": "熊"})

# batch
chain.batch([{"topic": "熊"}, {"topic": "猫"}], config={"max_concurrency": 5}) # 可设置并发数

# Async Stream 异步
async for s in chain.astream({"topic": "女人"}):
    print(s.content, end="", flush=True)

# Async Invoke
await chain.ainvoke({"topic": "男人"})

# Async Batch
await chain.abatch([{"topic": "熊"},{"topic": "女人"}])

## 异步获取中间步骤

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings

template = """基于下面的上下文来回答问题:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

vectorstore = FAISS.from_texts(
    ["柯基犬是一种中型家庭宠物犬"], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

retrieval_chain = (
    {
        "context": retriever.with_config(run_name="Docs"),
        "question": RunnablePassthrough(),
    }
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
async for chunk in retrieval_chain.astream_log(
    "柯基是什么?", include_names=["Docs"]
):
    print("-" * 40)
    print(chunk)

## 并行支持

In [ ]:
from langchain_core.runnables import RunnableParallel

chain1 = ChatPromptTemplate.from_template("给我讲一个关于{topic}的笑话") | model
chain2 = (
    ChatPromptTemplate.from_template("写两行关于{topic}的诗歌")
    | model
)
combined = RunnableParallel(joke=chain1, poem=chain2)

In [ ]:
%%time
combined.invoke({"topic": "熊"})

In [ ]:
# 并行批处理，适用于大量生成
%%time
combined.batch([{\"topic\": \"熊\"}, {\"topic\": \"猫\"}])